In [14]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, isnan, when, split, regexp_replace, to_date, year, month, dayofmonth, dayofweek, dayofyear, weekofyear, upper, translate, trim
import pandas as pd
import unicodedata
from schema import SCHEMA
from dataclean_car import Dataclean

from pyspark.sql import SparkSession
import boto3

ImportError: cannot import name 'SaveMode' from 'pyspark.sql' (/usr/local/spark/python/pyspark/sql/__init__.py)

In [5]:
# Retrieve AWS credentials from the EC2 instance metadata
credentials = boto3.Session().get_credentials()

# Configure SparkSession with S3 credentials
spark = SparkSession.builder \
    .appName("Read CSV from S3 Example") \
    .config("spark.hadoop.fs.s3a.access.key", credentials.access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", credentials.secret_key) \
    .config("spark.hadoop.fs.s3a.session.token", credentials.token) \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1") \
    .getOrCreate()

In [6]:
#file = "s3a://criminix-criminix-raw/car_theft/year=2005/month=07/2005-07-01.xlsx"
files = []
s3 = "s3a://criminix-criminix-raw/"

for year in range(2003,2023):
    for month in range(1,13):
        filename = f"{year}-{month:02d}-01.xlsx"
        path = f"{s3}car_theft/year={year}/month={month:02d}/{filename}"
        files.append(path)

df = spark.read.option("encoding", "utf-16le").option("sep", "\t").option("header", "true").option("multiLine", "true").csv(files, schema=SCHEMA)

df.count()

2574991

In [7]:
dtcl = Dataclean(df=df)
dtcl.drop_columns()
dtcl.to_upper_case()
dtcl.remove_accents()
dtcl.format_dates()
dtcl.cast_types()
dtcl.format_vehicle_brands_models()
dtcl.format_abstence_with_null()
dtcl.format_saint_names()
dtcl.format_street_name()
dtcl.format_cities_names()
dtcl.drop_duplicates()
dtcl.format_empty_values()

In [8]:
dfx = dtcl.get_df()
dfx.count()

1601839

In [16]:
s3_clean = "s3a://criminix-criminix-clean/"

path = f"{s3_clean}car_theft/car_theft_csv"
dfx.write.option("header", "true").mode("overwrite").csv(path)

#path = f"{s3_clean}car_theft/car_theft_parquet"
#dfx.write.option("header", "true").parquet(path)

In [17]:
dfx_latest = dfx.select(col("ANO").isin([2020, 2021, 2022]))

s3_clean = "s3a://criminix-criminix-clean/"

path = f"{s3_clean}car_theft/car_theft_latest_csv"
dfx_latest.coalesce(1).write.option("header", "true").mode("overwrite").csv(path)